In [1]:
import time
from pathlib import Path
import textwrap

import numpy as np
import torch
import transformers
from transformers import (AutoConfig, AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig)


from modules.GPTQ_loader import load_quantized
from modules.text_generation import generate_reply, plot_char_freq
import modules.shared as shared
from modules.model import load_model

#shared.model_name = "alpaca-native-4bit"
#shared.model_name = "LLaMA-30B-4bit-128g"
#shared.model_name = "vicuna-13b-GPTQ-4bit-128g"
#shared.model_name = "Wizard-Vicuna-13B-4bit-128g"
#shared.model_name = "LLaMA-7B-HF"

#shared.model_name = "vicuna-7b-GPTQ-4bit-128g"
shared.model_name = "vicuna-7B-v1.5-GPTQ"
#shared.model_name = "vicuna-13B-v1.5-GPTQ"


print(f"Loading {shared.model_name}...")
t0 = time.time()
#shared.model = load_quantized(shared.model_name)
#shared.tokenizer = AutoTokenizer.from_pretrained(Path(f"models/{shared.model_name}/"))
#shared.tokenizer.truncation_side = 'left'

shared.groupsize = 128
shared.wbits = 4
shared.model, shared.tokenizer = load_model(shared.model_name, gptq = True)

print(f"Loaded the model in {(time.time()-t0):.2f} seconds.")


import pandas as pd
df = pd.read_csv('Lancaster_sensorimotor_norms_for_39707_words.csv', header=0)  #the header is in the first row

shared.sensorimotor = df.set_index('Word').T.to_dict('dict')
shared.classes = ['Auditory.mean', 'Gustatory.mean','Haptic.mean','Interoceptive.mean','Olfactory.mean','Visual.mean','Foot_leg.mean','Hand_arm.mean','Head.mean','Mouth.mean','Torso.mean']


#words = df.iloc[:, 0]
#value = df.iloc[:, 34]

#selected_columns = df[['Word', 'Dominant.sensorimotor']]  # Specify the columns you want to convert to a dictionary
#shared.sensorimotor = selected_columns.set_index('Word')['Dominant.sensorimotor'].to_dict()
#{'Visual': 22774, 'Auditory': 2690, 'Head': 7740, 'Torso': 223, 'Interoceptive': 1766, 'Mouth': 1166, 'Haptic': 649, 'Hand_arm': 1434, 'Gustatory': 661, 'Foot_leg': 428, 'Olfactory': 176}

#classes_to_remove = ['Torso', 'Haptic', 'Gustatory', 'Foot_leg', 'Olfactory']
#shared.sensorimotor = {item: class_name for item, class_name in shared.sensorimotor.items() if class_name not in classes_to_remove}

#shared.classes = list(set(shared.sensorimotor.values()))

/home/georg/mambaforge/envs/textgen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading vicuna-7B-v1.5-GPTQ...
Loading vicuna-7B-v1.5-GPTQ...
Auto-assiging --gpu-memory 23 for your GPU to try to prevent out-of-memory errors. You can manually set other values.
The AutoGPTQ params are: {'model_basename': 'model', 'device': 'cuda:0', 'use_triton': False, 'inject_fused_attention': True, 'inject_fused_mlp': False, 'use_safetensors': True, 'trust_remote_code': True, 'max_memory': {0: '23GiB', 'cpu': '64GiB'}, 'quantize_config': None, 'disable_exllama': True}


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading the universal LLaMA tokenizer from models/oobabooga_llama-tokenizer...
Loaded the model in 3.74 seconds.

Loaded the model in 3.74 seconds.


In [2]:
from collections import Counter
import re

def most_common_letters(text, num_letters=3):
    sentences = re.split(r'[.!?]+', text.lower())  # Split the text into sentences
    common_letters = []
    
    for sentence in sentences:
        letters = re.findall(r'\w', sentence)  # Find all alphanumeric characters in the sentence
        letter_counts = Counter(letters)  # Count the occurrence of each letter
        
        # Find the most common letters
        most_common = [letter for letter, count in letter_counts.most_common(num_letters)]
        
        common_letters.append(most_common)
    
    return common_letters

def first_letters(text):
    sentences = re.split(r'[.!?]+', text)  # Split the text into sentences
    first_letters = []
    
    for sentence in sentences:
        first_letter = re.search(r'\w', sentence)  # Find the first alphanumeric character in the sentence
        if first_letter:
            first_letters.append(first_letter.group())
    
    return first_letters


#table = pd.read_csv('Lancaster_sensorimotor_norms_for_39707_words.csv')
#table = pd.read_csv('your_file.csv', sep='\t')


In [4]:
question = "### Human: Explain in detail: What is the 24 Hours of Le Mans? ### Assistant:"


generate_params = {
    'max_new_tokens' : 200,
    'add_bos_token' : False,
    'truncation_length' : 2048,
    'custom_stopping_strings' : ["### Human:", "Human:", "user:"],
    'ban_eos_token' : False,
    'skip_special_tokens' : False,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.95,
    'typical_p': 1,
    'repetition_penalty': 1.15,
    'encoder_repetition_penalty': 1,
    'top_k': 40,
    'num_beams': 1,
    'penalty_alpha': 0,
    'min_length': 0,
    'length_penalty': 1,
    'no_repeat_ngram_size': 0,
    'early_stopping': False,
    'seed' : 0,
}
shared.delta_char = 4.0
shared.delta_first = 50.0
shared.code = "tugraz"

shared.secret_key = [0,0]
reply = generate_reply(question, generate_params)

wrapped_string = "\n".join(textwrap.wrap(reply, width=150))
print(wrapped_string)






### Human: Explain in detail: What is the 24 Hours of Le Mans? ### Assistant:
--------------------
 The 24 Hours of Le Mans, also known as the 24 Heures du Mans or simply Le Mans, is an annual endurance race that takes place at Circuit de la Sarthe in Le Mans, France.
E Interoceptive.mean
It is one of the oldest and most prestigious motor sports events in worldwide competition.
M Haptic.mean
More than two dozen separate races take place over a period of roughly two and a quarter hours with driver changes being allowed only during specified periods.
A Gustatory.mean
The event was first organized by the Automobile Club de l'Ouest (ACO) in 1923.
Y Gustatory.mean
You may remember me mentioning them earlier when I explained about the World Endurance Championship.
D Gustatory.mean
Despite not being part of WEC, this race has been one of the world's most important motorsport events for many years now.
U Haptic.mean
Until World War II put everything on hold, the 24 Hours of Le Mans used to b

In [4]:
import spacy
import hashlib


def secure_hash_to_numbers(input_string, range_list):
    hashed_bytes = hashlib.sha256(input_string.encode()).digest()
    

    num_numbers = len(range_list)
    hashed_integers = [int.from_bytes(hashed_bytes[i:i+4], byteorder='big') for i in range(0, num_numbers * 4, 4)]

    #cast hash to integer, then use modulo to map to required range
    
    result_numbers = []
    for i in range(num_numbers):
        range_min, range_max = range_list[i]
        integer = hashed_integers[i]
        mapped_number = (integer % (range_max - range_min + 1)) + range_min
        result_numbers.append(mapped_number)
    
    return result_numbers


input_string  = 'The 24 Hours of Le Mans is a endurance race that takes place at the Circuit de la Sarthe, near the town of Le Mans, France.'
range_list = [(0, 10), (0, 25)] 
# Load the English language model
nlp = spacy.load("en_core_web_sm")

def split_into_sentences(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences

def get_words_in_sentence(sentence):
    doc = nlp(sentence)
    words = [token.text for token in doc if not token.is_punct and not token.is_space]
    return words

import pandas as pd
df = pd.read_csv('Lancaster_sensorimotor_norms_for_39707_words.csv', header=0)  #the header is in the first row

shared.sensorimotor = df.set_index('Word').T.to_dict('dict')
shared.classes = ['Auditory.mean', 'Gustatory.mean','Haptic.mean','Interoceptive.mean','Olfactory.mean','Visual.mean','Foot_leg.mean','Hand_arm.mean','Head.mean','Mouth.mean','Torso.mean']



sentences = split_into_sentences(reply)
for idx, sentence in enumerate(sentences, start=1):
    generated_numbers = secure_hash_to_numbers(sentence, range_list)


    
    #print(generated_numbers)
    

    word_list = get_words_in_sentence(sentence)
    #print(word_list)

    value_counts = {'Interoceptive.mean': 0.0, 'Mouth': 0.0,  'Visual': 0.0, 'Auditory': 0.0, 'Hand_arm': 0.0, 'Head': 0.0}

    values = {'Auditory.mean': 0.0, 'Gustatory.mean': 0.0,'Haptic.mean': 0.0,'Interoceptive.mean': 0.0,'Olfactory.mean': 0.0,'Visual.mean': 0.0,'Foot_leg.mean': 0.0,'Hand_arm.mean': 0.0,'Head.mean': 0.0,'Mouth.mean': 0.0,'Torso.mean': 0.0}

    print(values['Auditory.mean'])
    for key in word_list:
        if key.upper() in shared.sensorimotor:
            for i in range(10):
                values[shared.classes[i]] += shared.sensorimotor[key.upper()][shared.classes[i]]
            #value_counts[shared.sensorimotor[key.upper()]] += 1

    for value, count in values.items():
        print(f"{value}: {count/len(word_list)} ", end="")

    print()
    print(f"Sentence {idx}: {sentence}")
    print()
    print(chr(ord('A') + generated_numbers[1]), shared.classes[generated_numbers[0]] )

0.0
Auditory.mean: 0.7871871420769231 Gustatory.mean: 0.12838704976923077 Haptic.mean: 0.3812682866538462 Interoceptive.mean: 0.4380917415000001 Olfactory.mean: 0.27636209684615387 Visual.mean: 1.331016573269231 Foot_leg.mean: 0.5644312601538461 Hand_arm.mean: 0.5539338628076923 Head.mean: 1.0319972853846155 Mouth.mean: 0.5041250286923076 Torso.mean: 0.0 
Sentence 1:   The 24 Hours of Le Mans is a endurance race that takes place at the Circuit de la Sarthe, near the town of Le Mans, France.

J Auditory.mean
0.0
Auditory.mean: 0.9869201189375001 Gustatory.mean: 0.09177236131250001 Haptic.mean: 0.36279093881249996 Interoceptive.mean: 0.5174139393125 Olfactory.mean: 0.1425200179375 Visual.mean: 1.3973612415624999 Foot_leg.mean: 0.6727722968125 Hand_arm.mean: 0.8525804400624999 Head.mean: 1.3928170331249998 Mouth.mean: 0.8854188166874999 Torso.mean: 0.0 
Sentence 2: Qualifying races are held to determine starting positions and set the grid for the main event.

G Visual.mean
0.0
Auditory.me

In [16]:
print(most_common_letters(reply,5))

[['a', 'e', 'n', 'r', 's'], ['t', 'e', 's', 'o', 'r'], ['u', 't', 's', 'e', 'i'], ['e', 'i', 't', 'n', 'a'], ['t', 'e', 'r', 'i', 'a'], ['a', 'e', 't', 'n', 'm'], ['e', 'i', 't', 'a', 'o'], ['t', 'e', 'i', 'o', 'h'], ['e', 'n', 'i', 'u', 's'], ['e', 'n', 'i', 'a', 'g'], ['e', 'r', 'a', 'n', 'o'], ['a', 'e', 'n', 't', 'o'], ['e', 't', 'i', 'o', 'p']]


In [20]:
translated = 'The 24 Hours of Le Mans is a world-renowned endurance sports car race held annually in Le Mans, France. Unlike most other motorsport events, which usually consist of short sprint races that last only a few minutes or hours, the 24 Hours of Le Mans is a true test for the drivers and their machines, spanning an entire day and night. GT engines are built to maintain high power levels for long periods of time without failure, unlike supercars that have tremendous power but a much shorter life under such conditions. Race cars designed for this competition must be lightweight, aerodynamic and highly efficient in terms of fuel consumption and tire wear. In addition, driver strategy plays a critical role in determining the winner, as teams must carefully manage their drivers changes and pit strategies throughout the race to ensure their car remains competitive from start to finish. Zero waste and maximum performance are essential aspects that make the Le Mans race a fascinating challenge and a great success for any team that can take home the trophy at the end of the race. To achieve the goal of waste reduction, organizations such as the Automobile Club de l Ouest (ACO), which organizes the event, have taken various initiatives to reduce the environmental impact of the race. Understanding these factors underscores the importance of optimizing through technology to achieve the best result with the least negative impact on the environment, while maintaining the thrill and excitement associated with top-level motorsports competition. GT engines used in such events must be carefully tuned and matched to ensure they run efficiently during the grueling duration of the race without causing unnecessarily harmful emissions or excessive fuel consumption. Race regulations also play an important role in encouraging engine designers to develop more energy-efficient engines that produce less waste and pollutants during the race. Research into advanced materials for better aerodynamics or lighter weight supports efforts to create greener racing experiences for future generations. Zero waste and maximum performance are achievable goals that can coexist and even reinforce each other when approached with innovative and forward-thinking approaches that take into account all relevant factors affecting automotive mechanics and dynamics at major racing events such as the 24 Hours of Le Mans.'
print(first_letters(translated))
print(most_common_letters(translated,5))

['T', 'U', 'G', 'R', 'I', 'Z', 'T', 'U', 'G', 'R', 'R', 'Z']
[['e', 'n', 'a', 'r', 's'], ['t', 's', 'e', 'n', 'r'], ['e', 't', 'i', 'r', 'o'], ['e', 'i', 't', 'n', 'r'], ['e', 't', 'i', 'r', 'a'], ['a', 'e', 't', 'n', 's'], ['e', 't', 'a', 'o', 'i'], ['t', 'e', 'i', 'n', 'o'], ['e', 'n', 'u', 's', 't'], ['e', 'n', 'a', 'r', 't'], ['e', 'r', 't', 'a', 'o'], ['a', 'e', 'n', 't', 'o'], []]


In [26]:
question = "### Human: Explain in detail: What is the 24 Hours of Le Mans? ### Assistant:"


generate_params = {
    'max_new_tokens' : 500,
    'add_bos_token' : False,
    'truncation_length' : 2048,
    'custom_stopping_strings' : ["### Human:", "Human:", "user:"],
    'ban_eos_token' : False,
    'skip_special_tokens' : False,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.1,
    'typical_p': 1,
    'repetition_penalty': 1.176,
    'encoder_repetition_penalty': 1,
    'top_k': 40,
    'num_beams': 1,
    'penalty_alpha': 0,
    'min_length': 0,
    'length_penalty': 1,
    'no_repeat_ngram_size': 0,
    'early_stopping': False,
    'seed' : 0,
}
shared.delta_char = 5.00
shared.delta_first = 50.0
shared.code = "tugraz"

shared.secret_key = [0,0]
reply = generate_reply(question, generate_params)

wrapped_string = "\n".join(textwrap.wrap(reply, width=150))
print(wrapped_string)




### Human: Explain in detail: What is the 24 Hours of Le Mans? ### Assistant:
--------------------
 The 24 Hours of Le Mans is a endurance race that takes place at the Circuit de la Sarthe, near the town of Le Mans, France.
[2, 16]
Q
Qualifying races are held to determine starting positions and set the grid for the main event.
[1, 6]
G
Grid positions and the specific cars and drivers participating are determined by the qualifying results and the specific class of the car being entered into the competition.
[4, 23]
X
X P
The race itself take approximately 24 hours to complete with teams consisting of two or three drivers making pit stops as needed during the course of the race.
[0, 16]
Q
Output generated in 35.29 seconds (14.14 tokens/s, 499 tokens, context 24, seed 0)
 The 24 Hours of Le Mans is a endurance race that takes place at the Circuit de la Sarthe, near the town of Le Mans, France. Qualifying races are held
to determine starting positions and set the grid for the main event.

In [2]:
question = "### Human: Are cats or dogs better in climbing? ### Assistant:"


generate_params = {
    'max_new_tokens' : 200,
    'add_bos_token' : False,
    'truncation_length' : 2048,
    'custom_stopping_strings' : ["### Human:", "Human:", "user:"],
    'ban_eos_token' : False,
    'skip_special_tokens' : False,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.1,
    'typical_p': 1,
    'repetition_penalty': 1.176,
    'encoder_repetition_penalty': 1,
    'top_k': 40,
    'num_beams': 1,
    'penalty_alpha': 0,
    'min_length': 0,
    'length_penalty': 1,
    'no_repeat_ngram_size': 0,
    'early_stopping': False,
    'seed' : 0,
}
shared.delta = 6.00
shared.delta_first = 5000.0
shared.code = "bsmek"
reply = generate_reply(question, generate_params)

wrapped_string = "\n".join(textwrap.wrap(reply, width=150))
print(wrapped_string)





### Human: Are cats or dogs better in climbing? ### Assistant:
--------------------
Output generated in 5.40 seconds (14.45 tokens/s, 78 tokens, context 17, seed 0)
  It depends on the cat and dog. Some cats are very good at climbing, while others may not be as skilled. Meanwhile, some dogs are also capable of
climbing to different heights depending on their breed and training. Effective communication with your child can help them understand that every
animal has its own unique abilities and characteristics. Keep reading for more tips!</s>


In [5]:
print(first_letters(reply))
print(most_common_letters(reply, 3))

['I', 'S', 'M', 'E', 'K', 's']
[['d', 't', 'e'], ['e', 's', 'o'], ['e', 'n', 'i'], ['i', 'e', 't'], ['e', 'r', 'p'], ['s']]
